In [10]:
import numpy as np
import os
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from tqdm import tqdm
import matplotlib.pyplot as plt
from h5py import File
from scripts.dataset import Dataset
from scripts.model import STRCA, FBTRCA
from scipy.linalg import eigh
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin

In [29]:
if __name__ == "__main__":
    Fs = 256
    num_fold = 5
    num_subj = 15
    frange = np.array([2, 4, 6, 8, 10])
    channels = (np.int64([1, 3, 5, 15, 29, 30, 31, 45, 55, 57, 59]) - 1).tolist()
    t = [0, 2]
    result = np.zeros([num_subj, num_fold, 4])

    Co_values = np.arange(1, 12)  
    param_grid = {'Co': Co_values}

    for sub in range(num_subj):
        # dataset=Dataset(num_class=7,path='Dataset/DataBank/Dataset I/', channel=channels)
        dataset=Dataset(num_class=7,path='Dataset/OData/Dataset I/', channel=channels)
        dataset.load_data(sub, None)
        dataset.divide_data(num_fold)
        dataset.filter_data(fs=Fs, frange=frange, order=8, btype='lowpass')
        for fold in tqdm(range(num_fold), desc=f'Sub{sub}:'):
            train_set, train_label, test_set, test_label = dataset.retreive_data(
                    class_index=[0,1,2,3,4,5,6], fold_index=fold, filter_index=np.arange(len(frange))+1)
            train_set = train_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]
            test_set  = test_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]

            
            best_score = 0
            best_Co = None
            for Co in Co_values:
                fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=False)  
                fbtrca.fit(train_set, train_label)
                train_data = fbtrca.transform(train_set).reshape((len(train_label), -1))
                svm = SVC(kernel='rbf')
                svm.fit(train_data, train_label)
                score = svm.score(train_data, train_label)  
                if score > best_score:
                    best_score = score
                    best_Co = Co

            
            fbtrca = FBTRCA(best_Co, frange, optimizer='tdlda', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K * 8 * len(frange))), train_label)
            result[sub, fold, 2] = svm.score(test_data.reshape((len(test_label), fbtrca.K * 8 * len(frange))), test_label)

            
            fbtrca = FBTRCA(best_Co, frange, optimizer='tdlda', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K * 8 * len(frange))), train_label)
            result[sub, fold, 3] = svm.score(test_data.reshape((len(test_label), fbtrca.K * 8 * len(frange))), test_label)

    
    print("最佳平均准确率 ifG=False:", np.mean(result[:, :, 2], axis=(0, 1)))
    print("最佳平均准确率 ifG=True:", np.mean(result[:, :, 3], axis=(0, 1)))

Sub0::  20%|██        | 1/5 [00:25<01:41, 25.29s/it]


KeyboardInterrupt: 

In [21]:
np.mean(result,axis=(0,1))

array([0.        , 0.        , 0.4460796 , 0.45153193])

In [3]:
if __name__=="__main__":
    Fs = 256
    Co = 21
    num_fold = 5
    num_subj = 15
    max_freq = 10
    step_bank= 2
    frange = np.arange(step_bank,max_freq+1,step_bank)
    channels = (np.int64([1,3,5,12,15,18,20,21,24,25,29,30,31,35,36,39,40,45,55,57,59])-1).tolist()
    num_bank = int(max_freq/step_bank)
    t  = [0, 2]
    result = np.zeros([num_subj, num_fold, 4])

    for sub in range(num_subj):
        # dataset=Dataset(num_class=7,path='Dataset/DataBank/Dataset I/', channel=channels)
        dataset=Dataset(num_class=7,path='Dataset/OData/Dataset I/', channel=channels)
        dataset.load_data(sub, None)
        dataset.divide_data(num_fold)
        dataset.filter_data(fs=Fs, frange=frange, order=8, btype='lowpass')
        for fold in tqdm(range(num_fold), desc=f'Sub{sub}:'):
            train_set, train_label, test_set, test_label = dataset.retreive_data(
                    class_index=[0,1,2,3,4,5,6], fold_index=fold, filter_index=np.arange(len(frange))+1)
            train_set = train_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]
            test_set  = test_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 0] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 1] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)
            
            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 2] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 3] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

Sub14:: 100%|██████████| 5/5 [01:41<00:00, 20.36s/it]


In [4]:
np.mean(result,axis=(0,1))

array([0.44732911, 0.44591979, 0.46107003, 0.45626527])

In [3]:
if __name__=="__main__":
    Fs = 256
    Co = 31
    num_fold = 5
    num_subj = 15
    max_freq = 10
    step_bank= 2
    frange = np.arange(step_bank,max_freq+1,step_bank)
    channels = (np.int64([1,3,5,6,11,12,13,15,17,18,20,21,24,25,27,29,30,31,33,35,36,39,40,42,43,45,47,48,55,57,59])-1).tolist()
    num_bank = int(max_freq/step_bank)
    t  = [0, 2]
    result = np.zeros([num_subj, num_fold, 4])

    for sub in range(num_subj):
        # dataset=Dataset(num_class=7,path='Dataset/DataBank/Dataset I/', channel=channels)
        dataset=Dataset(num_class=7,path='Dataset/OData/Dataset I/', channel=channels)
        dataset.load_data(sub, None)
        dataset.divide_data(num_fold)
        dataset.filter_data(fs=Fs, frange=frange, order=8, btype='lowpass')
        for fold in tqdm(range(num_fold), desc=f'Sub{sub}:'):
            train_set, train_label, test_set, test_label = dataset.retreive_data(
                    class_index=[0,1,2,3,4,5,6], fold_index=fold, filter_index=np.arange(len(frange))+1)
            train_set = train_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]
            test_set  = test_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 0] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 1] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)
            
            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 2] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 3] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

Sub14:: 100%|██████████| 5/5 [02:51<00:00, 34.22s/it]


In [4]:
np.mean(result,axis=(0,1))

array([0.46253791, 0.45556356, 0.47139584, 0.47228209])

In [5]:
if __name__=="__main__":
    Fs = 256
    Co = 41
    num_fold = 5
    num_subj = 15
    max_freq = 10
    step_bank= 2
    frange = np.arange(step_bank,max_freq+1,step_bank)
    channels = (np.int64([1,3,5,6,7,10,11,12,13,15,17,18,19,20,21,24,25,26,27,28,29,30,31,32,33,35,36,39,40,42,43,45,47,48,49,50,53,54,55,57,59])-1).tolist()
    num_bank = int(max_freq/step_bank)
    t  = [0, 2]
    result = np.zeros([num_subj, num_fold, 4])

    for sub in range(num_subj):
        # dataset=Dataset(num_class=7,path='Dataset/DataBank/Dataset I/', channel=channels)
        dataset=Dataset(num_class=7,path='Dataset/OData/Dataset I/', channel=channels)
        dataset.load_data(sub, None)
        dataset.divide_data(num_fold)
        dataset.filter_data(fs=Fs, frange=frange, order=8, btype='lowpass')
        for fold in tqdm(range(num_fold), desc=f'Sub{sub}:'):
            train_set, train_label, test_set, test_label = dataset.retreive_data(
                    class_index=[0,1,2,3,4,5,6], fold_index=fold, filter_index=np.arange(len(frange))+1)
            train_set = train_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]
            test_set  = test_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 0] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 1] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)
            
            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 2] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 3] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

Sub14:: 100%|██████████| 5/5 [04:38<00:00, 55.63s/it]


In [6]:
np.mean(result,axis=(0,1))

array([0.46018007, 0.4562164 , 0.47207057, 0.46807405])

In [7]:
if __name__=="__main__":
    Fs = 256
    Co = 51
    num_fold = 5
    num_subj = 15
    max_freq = 10
    step_bank= 2
    frange = np.arange(step_bank,max_freq+1,step_bank)
    channels = (np.int64([1,2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,24,25,26,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,46,47,48,49,50,53,54,55,56,57,58,59])-1).tolist()
    num_bank = int(max_freq/step_bank)
    t  = [0, 2]
    result = np.zeros([num_subj, num_fold, 4])

    for sub in range(num_subj):
        # dataset=Dataset(num_class=7,path='Dataset/DataBank/Dataset I/', channel=channels)
        dataset=Dataset(num_class=7,path='Dataset/OData/Dataset I/', channel=channels)
        dataset.load_data(sub, None)
        dataset.divide_data(num_fold)
        dataset.filter_data(fs=Fs, frange=frange, order=8, btype='lowpass')
        for fold in tqdm(range(num_fold), desc=f'Sub{sub}:'):
            train_set, train_label, test_set, test_label = dataset.retreive_data(
                    class_index=[0,1,2,3,4,5,6], fold_index=fold, filter_index=np.arange(len(frange))+1)
            train_set = train_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]
            test_set  = test_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 0] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 1] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)
            
            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 2] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 3] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

Sub14:: 100%|██████████| 5/5 [06:50<00:00, 82.04s/it]


In [8]:
np.mean(result,axis=(0,1))

array([0.45614452, 0.45405769, 0.46735979, 0.46336343])

In [9]:
if __name__=="__main__":
    Fs = 256
    Co = 61
    num_fold = 5
    num_subj = 15
    max_freq = 10
    step_bank= 2
    frange = np.arange(step_bank,max_freq+1,step_bank)
    channels = (np.int64([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61])-1).tolist()
    num_bank = int(max_freq/step_bank)
    t  = [0, 2]
    result = np.zeros([num_subj, num_fold, 4])

    for sub in range(num_subj):
        # dataset=Dataset(num_class=7,path='Dataset/DataBank/Dataset I/', channel=channels)
        dataset=Dataset(num_class=7,path='Dataset/OData/Dataset I/', channel=channels)
        dataset.load_data(sub, None)
        dataset.divide_data(num_fold)
        dataset.filter_data(fs=Fs, frange=frange, order=8, btype='lowpass')
        for fold in tqdm(range(num_fold), desc=f'Sub{sub}:'):
            train_set, train_label, test_set, test_label = dataset.retreive_data(
                    class_index=[0,1,2,3,4,5,6], fold_index=fold, filter_index=np.arange(len(frange))+1)
            train_set = train_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]
            test_set  = test_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 0] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 1] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)
            
            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 2] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 3] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

Sub14:: 100%|██████████| 5/5 [09:17<00:00, 111.42s/it]


In [10]:
np.mean(result,axis=(0,1))

array([0.4584882 , 0.45399375, 0.47530588, 0.46382434])